In [10]:
import spacy
from spacy import displacy
import re
nlp=spacy.load('en_core_web_trf')

In [40]:
corpus=[]
with open("./data/dataset.txt") as file:
    sentence=file.readline()
    while sentence:
        corpus+=[sentence]
        sentence=file.readline()
corpus=[re.sub(" <[\S]*> "," ",s) for s in corpus]
corpus=[re.sub(" \. ",". ",s) for s in corpus]
corpus=[re.sub(" , ",", ",s) for s in corpus]

In [42]:
doc=nlp(corpus[0])
displacy.render(doc, style='dep', jupyter='True')

E:\Anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [41]:
print(corpus[0])

Tehran's seismologic center said the quake measured magnitude - 5. 9, but the U. S. Geological Survey in Golden, Colo., said it was a magnitude  - 6. 1  temblor .

